In [ ]:
# !pip3 install matplotlib
# !pip3 install tabula-py
# !pip3 install PyPDF2
# !pip3 install pdfminer.six
# !pip3 install PyMuPDF
# !pip3 install camelot-py
# !pip3 install --upgrade pip
# !pip3 install geopandas
# !pip3 install plotly
# !pip3 install pandas
# !pip3 install pdfplumber
# !pip3 install scipy
# !pip3 install --upgrade pandas
# %pip install arcgis
# !pip3 install json
# !pip3 install gtfs_functions
# !pip3 install keplergl
# %pip install Ipython

In [313]:
import pandas as pd
import numpy as np
import matplotlib as mpt
import os
import geopandas as gpd
import fiona
from glob import glob
import plotly.express as px
from shapely.geometry import Point, MultiPolygon
from arcgis.gis import GIS
from arcgis.features.analysis import summarize_within
from IPython.display import display
from gtfs_functions import Feed
from gtfs_functions.gtfs_plots import map_gdf
# import keplergl as kpl


In [ ]:

def save_to_csv(df, output_folder, file_name):
    """AI is creating summary for save_to_csv

    Args:
        df (Pandas Data Frame): Contains data frame that needs to be converted and saved as csv
        output_folder (Str): Destination path where df needs to be saved
        file_name (Str): File name that df needs to be saved as
    """
    # Check if the output folder exists, if not, create it
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Define the output file path
    output_file_path = os.path.join(output_folder, file_name)
    
    # Save the DataFrame to CSV in the output folder
    df.to_csv(output_file_path, index=False)
    print(f"DataFrame saved to: {output_file_path}")


In [ ]:
def convert_data (file_path):
    """
    Function to convert files of appropriate structure to csv format, creating point object with files that have lat and lon values

    Args:
        file_path (str): File Path to open and convert to csv file format
    """
    df = pd.read_csv(file_path)
    output_folder_path = os.getcwd()+"/Data Sets/Analysis Data/"
    # print(output_folder_path)
    output_file_name = file_path.split('/')[-1].split('.')[0] + ".csv"
    save_to_csv(df, output_folder_path, output_file_name)
    print("{} saved in {}".format(output_file_name, output_folder_path))

In [325]:
def assign_grade(score):
    for grade, boundary in grade_boundaries.items():
        if score >= boundary:
            return grade
    return 'F'

In [437]:
def assign_time(time_list):
    time_format_list = [t+':00' for t in time_list]
    return time_format_list

In [441]:
t = input("Input time range and separate by comma (eg. 6am-9am: 6,7,8,9):")
t_l = t.split(',')
t_list = assign_time(t_l)
print(t_list)

['6:00', '7:00']


<function __main__.assign_time(time_list)>

In [ ]:
data_path = os.getcwd()+"/Data Sets/CT_GTFS/"
# ignore_file = ['agency.txt', "calendar.txt","calendar_dates.txt"]
convert_data(data_path+"shapes.txt")

# Program To Calculate Demand Index Using Z-Method

In [ ]:
community_data_path = os.getcwd()+"/Data Sets/Analysis Data/Community Profiles Compiled.xlsx"

In [ ]:
comm_df = pd.read_excel(community_data_path)
# comm_df.head()

In [ ]:
comm_df.head()

In [ ]:
output_folder_path = os.getcwd()+"/Data Sets/Analysis Data"

In [ ]:
#Z-Score method manually
pop_mean = comm_df['Population in private households'].mean()
pop_std = comm_df['Population in private households'].std()


medinc_mean = comm_df["Median household income of private households"].mean()
medinc_std = comm_df["Median household income of private households"].std()

low_mean = comm_df['Population in private households to whom low income concepts are applicable (Number in low income)'].mean()
low_std = comm_df['Population in private households to whom low income concepts are applicable (Number in low income)'].std()

trans_mean = ((comm_df['Employed']/comm_df["Employed labour force aged 15 years and over in private households"])*comm_df['Public transit']).mean()
trans_std = ((comm_df['Employed']/comm_df["Employed labour force aged 15 years and over in private households"])*comm_df['Public transit']).std()

rent_mean = (comm_df['Per cent households with income spending 30% or more total income on shelter (Renter)']*comm_df['Private households with total income greater than zero (Renter)']).mean()
rent_std = (comm_df['Per cent households with income spending 30% or more total income on shelter (Renter)']*comm_df['Private households with total income greater than zero (Renter)']).std()

seniors_mean = comm_df['65 to 84 years'].mean()
seniors_std = comm_df['65 to 84 years'].std()

In [ ]:
row, col = comm_df.shape
z_rows_to_append = []
for c in range(row):
    community = comm_df['Community Name'][c]
    
    pop = comm_df['Population in private households'][c]
    # pop_ss = (pop-pop_max)/(pop_max-pop_min)
    z_pop_score = (pop-pop_mean)/pop_std
    
    med = comm_df['Median household income of private households'][c]
    # med_ss = (med-medinc_max)/(medinc_max-medinc_min)
    z_med_score = (med-medinc_mean)/medinc_std
    
    trans = ((comm_df['Employed']/comm_df["Employed labour force aged 15 years and over in private households"])*comm_df['Public transit'])[c]
    # trans_ss = (trans-trans_max)/(trans_max-trans_min)
    z_trans_score = (trans-trans_mean)/trans_std
    
    rent = (comm_df['Per cent households with income spending 30% or more total income on shelter (Renter)']*comm_df['Private households with total income greater than zero (Renter)'])[c]
    # rent_ss = (rent-rent_max)/(rent_max-rent_min)
    z_rent_score = (rent-rent_mean)/rent_std
    
    low = comm_df['Population in private households to whom low income concepts are applicable (Number in low income)'][c]
    # low_ss = (low-low_max)/(low_max-low_min)
    z_low_score = (low-low_mean)/low_std

    sen = comm_df['65 to 84 years'][c]
    z_seniors_score = (sen-seniors_mean)/seniors_std

    z_result = z_trans_score+z_rent_score+z_low_score-z_med_score+z_seniors_score
    # z_result = z_trans_score+z_rent_score+z_low_score+z_seniors_score

    column_df = {"Community Name": community, "Z Score": z_result}
    columns_df = {"Community Name": community, "Low Income Index": z_low_score, "Seniors Index": z_seniors_score, "Public Transit Index": z_trans_score, "Rent Index": z_rent_score, "Median Income": z_med_score}
    z_rows_to_append.append(column_df)


Z_index_df = pd.DataFrame(z_rows_to_append)

In [ ]:
Z_index_df.head()

In [ ]:
z_max = Z_index_df['Z Score'].max()
z_min = Z_index_df['Z Score'].min()

In [ ]:
print(z_max)
print(z_min)

In [ ]:
row, col = Z_index_df.shape
z_ss_rows = []
for c in range(row):
    community = Z_index_df["Community Name"][c]
    z_score = Z_index_df['Z Score'][c]

    z_ss_result = (z_score-z_min)/(z_max-z_min)

    column_df = {"Community Name": community, "Z Score": z_ss_result}
    z_ss_rows.append(column_df)
    
z_ss_index_df = pd.DataFrame(z_ss_rows)

In [ ]:
z_ss_index_df.head()

In [ ]:
save_to_csv(z_ss_index_df, output_folder_path, 'Z Standardized Demand Index.csv')

# Generate Data Sets For Calculating Supply Frequency

In [422]:
gtfs_path = os.getcwd()+"/Data Sets/CT_GTFS.zip"
feed = Feed(gtfs_path, time_windows=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24])
# feed = Feed(gtfs_path, start_date = "2024-02-19", end_date = "2024-02-23", time_windows=[0,5,6,9,15,18,24])

In [423]:
feed.get_calendar()

INFO:root:Reading "calendar.txt".


,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,2023DE-1BUSWK-Weekday-02,1,1,1,1,1,0,0,20240117,20240202
1,2023DE-1BUSWK-Weekday-02-0000100,0,0,0,0,1,0,0,20240117,20240202
2,2023DE-1BUSWK-Weekday-02-1111000,1,1,1,1,0,0,0,20240117,20240202
3,2023DE-1BUSSA-Saturday-02,0,0,0,0,0,1,0,20240120,20240203
4,2023DE-1BUSSU-Sunday-02,0,0,0,0,0,0,1,20240121,20240204
5,2023DE-1BUSWK-Weekday-03,1,1,1,1,1,0,0,20240205,20240315
6,2023DE-1BUSWK-Weekday-03-0000100,0,0,0,0,1,0,0,20240205,20240315
7,2023DE-1BUSWK-Weekday-03-1111000,1,1,1,1,0,0,0,20240205,20240315
8,2023DE-1BUSSA-Saturday-03,0,0,0,0,0,1,0,20240210,20240316
9,2023DE-1BUSSU-Sunday-03,0,0,0,0,0,0,1,20240211,20240317


In [424]:
routes = feed.routes
trips = feed.trips
stops = feed.stops
stop_times = feed.stop_times
shapes = feed.shapes

INFO:root:Reading "routes.txt".
INFO:root:accessing trips
INFO:root:Start date is None. You should either specify a start date or set busiest_date to True.
INFO:root:Reading "trips.txt".
INFO:root:Reading "calendar.txt".
INFO:root:Reading "calendar_dates.txt".
INFO:root:The busiest date/s of this feed or your selected date range is/are:  ['2024-02-19'] with 17641 trips.
INFO:root:In the case that more than one busiest date was found, the first one will be considered.
INFO:root:In this case is 2024-02-19.
INFO:root:Reading "stop_times.txt".
INFO:root:_trips is defined in stop_times
INFO:root:Reading "stops.txt".
INFO:root:computing patterns
INFO:root:Reading "shapes.txt".


In [425]:
line_freq = feed.lines_freq

In [426]:
# line_freq.loc[line_freq["route_name"].str.contains("Line|MAX")].sort_values(by=['window']).to_excel("Lines_Frequency.xlsx")
ptn = line_freq.loc[line_freq["route_name"].str.contains("Line|MAX")].sort_values(by=['window'])

In [429]:
morning_rush = '6:00-9:00'
time_range = ['6:00','7:00','8:00','9:00','15:00','16:00','17:00','18:00']
pattern = '|'.join(time_range)
evening_rush = '15:00-18:00'

In [430]:
rush_hour_performance = ptn.loc[ptn['window'].str.contains(pattern)].sort_values(by=['route_name'])

In [431]:
ptn.shape

(357, 7)

In [432]:
# sum_trips = ptn.groupby('route_name')['ntrips'].sum()
n_trips = rush_hour_performance.groupby('route_id')['ntrips'].sum()

In [433]:
n_trips

route_id
201-20718    381
202-20718    324
303-20727    161
304-20727    119
306-20727    102
307-20727    112
Name: ntrips, dtype: int64

In [434]:
mean_min_per_trip = rush_hour_performance.groupby('route_id')['min_per_trip'].mean()
median_min_per_trip = rush_hour_performance.groupby('route_id')['min_per_trip'].median()

# SF SC Through Stop Sorted

In [191]:
data_path = os.getcwd()+"/Data Sets/Analysis Data/"
current_stops_data = "Stops Sorted.xlsx"
future_stops_data = "Stops Sorted Future Network.xlsx"
# comm_file = "Community Profiles Compiled.xlsx"
comm_file = 'demand.csv'

In [192]:
red_code = '201-20718'
blue_code = '202-20718'
orange_code = '303-20727'
yellow_code = '304-20727'
teal_code = '306-20727'
purple_code = '307-20727'

In [193]:
blue_trips = n_trips.loc[blue_code]
blue_freq = median_min_per_trip.loc[blue_code]
red_trips = n_trips.loc[red_code]
red_freq = median_min_per_trip.loc[red_code]
orange_trips = n_trips.loc[orange_code]
orange_freq = median_min_per_trip.loc[orange_code]
teal_trips = n_trips.loc[teal_code]
teal_freq = median_min_per_trip.loc[teal_code]
yellow_trips = n_trips.loc[yellow_code]
yellow_freq = median_min_per_trip.loc[yellow_code]
purple_trips = n_trips.loc[purple_code]
purple_freq = median_min_per_trip.loc[purple_code]

bus_capacity = 65
train_capacity = 600


In [194]:
comm_data = pd.read_csv(data_path+comm_file)
stop_df = pd.read_excel(data_path+current_stops_data, sheet_name = "Sheet2")

In [195]:
stop_df.head()

,Stops,Coordinates,Line,Community
0,69th street,"51.037317,-114.187902",Blue Line,Christie Park\nSpringbank Hill\nSignal Hill\nA...
1,Sirocco,"51.038353, -114.169128",Blue Line,Christie Park\nSignal Hill
2,45th Street,"51.03776,-114.153425",Blue Line,Westgate\nRosscarrock\nGlendale
3,Westbrook,"51.041596,-114.124162",Blue Line,Shaganappi\nRosscarrock\nKillarney
4,Shagannappi Point,"51.041724,-114.124038",Blue Line,Shaganappi


In [196]:
stop_df['Community'] = stop_df['Community'].str.rstrip('\n').str.replace('\n',',')

In [197]:
stop_df.head()

,Stops,Coordinates,Line,Community
0,69th street,"51.037317,-114.187902",Blue Line,"Christie Park,Springbank Hill,Signal Hill,Aspe..."
1,Sirocco,"51.038353, -114.169128",Blue Line,"Christie Park,Signal Hill"
2,45th Street,"51.03776,-114.153425",Blue Line,"Westgate,Rosscarrock,Glendale"
3,Westbrook,"51.041596,-114.124162",Blue Line,"Shaganappi,Rosscarrock,Killarney"
4,Shagannappi Point,"51.041724,-114.124038",Blue Line,Shaganappi


In [198]:
# service_dict = {}
comm_list = []
red_list = []
blue_list = []
orange_list = []
yellow_list = []
purple_list = []
teal_list = []

In [199]:
for index, row in stop_df.iterrows():
    if 'Line' in row['Line']:
        if 'Red' in row['Line']:
            red_list.extend([com for com in row['Community'].split(',')])
        else:
            blue_list.extend([com for com in row['Community'].split(',')])
    else:
        if 'Orange' in row['Line']:
            orange_list.extend([com for com in row['Community'].split(',')])
        elif 'Purple' in row['Line']:
            purple_list.extend([com for com in row['Community'].split(',')])
        elif 'Teal' in row['Line']:
            teal_list.extend([com for com in row['Community'].split(',')])
        else:
            yellow_list.extend([com for com in row['Community'].split(',')])

In [200]:
comm_list = list(set(red_list+blue_list+orange_list+purple_list+teal_list+yellow_list))
clean_com_list = [item for item in comm_list if item.strip()]
service_df = pd.DataFrame(clean_com_list, columns=["Community"]).sort_values(by="Community")

In [201]:
service_df

,Community
21,Acadia
38,Altadore
54,Applewood Park
65,Arbour Lake
26,Aspen Woods
...,...
93,Whitehorn
74,Willow Park
0,Winston Heights/Mountview
1,Woodbine


In [202]:
service_df["Red Trips"] = service_df["Community"].isin(red_list) * red_trips
service_df["Red Freq"] = service_df["Community"].isin(red_list) * red_freq
service_df["Blue Trips"] = service_df["Community"].isin(blue_list) * blue_trips
service_df["Blue Freq"] = service_df["Community"].isin(blue_list) * blue_freq
service_df["Orange Trips"] = service_df["Community"].isin(orange_list) * orange_trips
service_df["Orange Freq"] = service_df["Community"].isin(orange_list) * orange_freq
service_df["Yellow Trips"] = service_df["Community"].isin(yellow_list) * yellow_trips
service_df["Yellow Freq"] = service_df["Community"].isin(yellow_list) * yellow_freq
service_df["Teal Trips"] = service_df["Community"].isin(teal_list) * teal_trips
service_df["Teal Freq"] = service_df["Community"].isin(teal_list) * teal_freq
service_df["Purple Trips"] = service_df["Community"].isin(purple_list) * purple_trips
service_df["Purple Freq"] = service_df["Community"].isin(purple_list) * purple_freq

In [203]:
service_df

,Community,Red Trips,Red Freq,Blue Trips,Blue Freq,Orange Trips,Orange Freq,Yellow Trips,Yellow Freq,Teal Trips,Teal Freq,Purple Trips,Purple Freq
21,Acadia,226,10.0,0,0.0,0,0.0,0,0.0,61,12.0,0,0.0
38,Altadore,0,0.0,0,0.0,0,0.0,75,9.5,0,0.0,0,0.0
54,Applewood Park,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,66,25.0
65,Arbour Lake,226,10.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
26,Aspen Woods,0,0.0,193,4.0,0,0.0,0,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Whitehorn,0,0.0,193,4.0,98,7.0,0,0.0,0,0.0,0,0.0
74,Willow Park,226,10.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
0,Winston Heights/Mountview,0,0.0,0,0.0,98,7.0,0,0.0,0,0.0,0,0.0
1,Woodbine,0,0.0,0,0.0,0,0.0,75,9.5,0,0.0,0,0.0


In [204]:
# Merge the two DataFrames on the 'Community' and 'Community Name' columns
merged_data = pd.merge(service_df, comm_data, left_on='Community', right_on='Community Name', how='right')

# Update the 'Population' column in sup_fc with the corresponding values from comm_data
service_df['Population'] = service_df['Community'].apply(
    lambda x: merged_data.loc[merged_data['Community Name'] == x, 'Population in private households'].iloc[0]
    if x in merged_data['Community Name'].tolist() else 0
)

# dropping rows that have no population:
service_df = service_df[service_df['Population'] != 0]

In [205]:
service_df["Frequency Index"] = service_df.filter(like="Freq").sum(axis=1)
service_df['Total Trips'] = service_df.filter(regex="Red Trips|Blue Trips").sum(axis=1)*train_capacity + service_df.filter(regex="Orange Trips|Yellow Trips|Teal Trips|Yellow Trips|Purple Trips").sum(axis=1)*bus_capacity
service_df["Capacity Index"] = service_df['Total Trips']/service_df.filter(like='Population').sum(axis=1)

In [206]:
min_freq = min(service_df['Frequency Index'])
max_freq = max(service_df['Frequency Index'])
min_cap = min(service_df['Capacity Index'])
max_cap = max(service_df['Capacity Index'])

In [207]:
service_df['SS Frequency Index'] = (service_df['Frequency Index']-min_freq)/(max_freq-min_freq)
service_df['SS Capacity Index'] = (service_df['Capacity Index']-min_cap)/(max_cap-min_cap)

In [208]:
service_df

,Community,Red Trips,Red Freq,Blue Trips,Blue Freq,Orange Trips,Orange Freq,Yellow Trips,Yellow Freq,Teal Trips,Teal Freq,Purple Trips,Purple Freq,Population,Frequency Index,Total Trips,Capacity Index,SS Frequency Index,SS Capacity Index
21,Acadia,226,10.0,0,0.0,0,0.0,0,0.0,61,12.0,0,0.0,9915,22.0,139565,14.076147,0.404494,0.062027
38,Altadore,0,0.0,0,0.0,0,0.0,75,9.5,0,0.0,0,0.0,7290,9.5,4875,0.668724,0.123596,0.001630
54,Applewood Park,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,66,25.0,6830,25.0,4290,0.628111,0.471910,0.001447
65,Arbour Lake,226,10.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,10335,10.0,135600,13.120464,0.134831,0.057722
26,Aspen Woods,0,0.0,193,4.0,0,0.0,0,0.0,0,0.0,0,0.0,9435,4.0,115800,12.273450,0.000000,0.053906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Whitehorn,0,0.0,193,4.0,98,7.0,0,0.0,0,0.0,0,0.0,11085,11.0,122170,11.021200,0.157303,0.048265
74,Willow Park,226,10.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,5090,10.0,135600,26.640472,0.134831,0.118626
0,Winston Heights/Mountview,0,0.0,0,0.0,98,7.0,0,0.0,0,0.0,0,0.0,3605,7.0,6370,1.766990,0.067416,0.006577
1,Woodbine,0,0.0,0,0.0,0,0.0,75,9.5,0,0.0,0,0.0,8745,9.5,4875,0.557461,0.123596,0.001129


In [209]:
service_df.loc[service_df['SS Frequency Index'] == 0].shape

(10, 19)

In [210]:
service_df.loc[service_df['Population'] == 0].shape

(0, 19)

## Future stops

In [211]:
# service_dict = {}
comm_list = []
red_list = []
blue_list = []
orange_list = []
yellow_list = []
purple_list = []
teal_list = []
green_list = []
future_brentwood_list = []
future_tuscany_list = []
east_grey_list = []
west_grey_list = []

In [212]:
green_trips = n_trips.loc[red_code]
green_freq = median_min_per_trip.loc[red_code]
east_grey_trips = n_trips.loc[teal_code]
east_grey_freq = median_min_per_trip.loc[teal_code]
west_grey_trips = n_trips.loc[teal_code]
west_grey_freq = median_min_per_trip.loc[teal_code]
future_brentwood_trips = n_trips.loc[teal_code]
future_brentwood_freq = median_min_per_trip.loc[teal_code]
future_tuscany_trips = n_trips.loc[teal_code]
future_tuscany_freq = median_min_per_trip.loc[teal_code]

In [213]:
fut_stop_df = pd.read_excel(data_path+future_stops_data, sheet_name = "Sheet2")
fut_stop_df = fut_stop_df.dropna()

In [214]:
fut_stop_df['Community'] = fut_stop_df['Community'].str.rstrip('\n').str.replace('\n',',')

In [215]:
for index, row in fut_stop_df.iterrows():
    if 'Line' in row['Line']:
        if 'Red' in row['Line']:
            red_list.extend([com for com in row['Community'].split(',')])
        elif 'Blue' in row['Line']:
            blue_list.extend([com for com in row['Community'].split(',')])
        else:
            green_list.extend([com for com in row['Community'].split(',')])
    else:
        if 'Orange' in row['Line']:
            orange_list.extend([com for com in row['Community'].split(',')])
        elif 'Purple' in row['Line']:
            purple_list.extend([com for com in row['Community'].split(',')])
        elif 'Teal' in row['Line']:
            teal_list.extend([com for com in row['Community'].split(',')])
        elif 'Yellow' in row['Line']:
            yellow_list.extend([com for com in row['Community'].split(',')])
        elif 'East' in row['Line']:
            east_grey_list.extend([com for com in row['Community'].split(',')])
        elif 'West' in row['Line']:
            west_grey_list.extend([com for com in row['Community'].split(',')])
        elif 'Brentwood' in row['Line']:
            future_brentwood_list.extend([com for com in row['Community'].split(',')])
        elif 'Tuscany' in row['Line']:
            # print(row['Community'])
            future_tuscany_list.extend([com for com in row['Community'].split(',')])

In [216]:
fut_comm_list = list(set(red_list+blue_list+orange_list+purple_list+teal_list+yellow_list+green_list+east_grey_list+west_grey_list+future_brentwood_list+future_tuscany_list))
clean_com_list = [item for item in fut_comm_list if item.strip()]
fut_service_df = pd.DataFrame(clean_com_list, columns=["Community"]).sort_values(by="Community")

In [217]:
fut_service_df.head()

,Community
29,Acadia
73,Airport
52,Altadore
79,Applewood Park
95,Arbour Lake


In [218]:
fut_service_df["Red Trips"] = fut_service_df["Community"].isin(red_list) * red_trips
fut_service_df["Red Freq"] = fut_service_df["Community"].isin(red_list) * red_freq
fut_service_df["Blue Trips"] = fut_service_df["Community"].isin(blue_list) * blue_trips
fut_service_df["Blue Freq"] = fut_service_df["Community"].isin(blue_list) * blue_freq
fut_service_df["Green Trips"] = fut_service_df["Community"].isin(green_list) * green_trips
fut_service_df["Green Freq"] = fut_service_df["Community"].isin(green_list) * green_freq
fut_service_df["Orange Trips"] = fut_service_df["Community"].isin(orange_list) * orange_trips
fut_service_df["Orange Freq"] = fut_service_df["Community"].isin(orange_list) * orange_freq
fut_service_df["Yellow Trips"] = fut_service_df["Community"].isin(yellow_list) * yellow_trips
fut_service_df["Yellow Freq"] = fut_service_df["Community"].isin(yellow_list) * yellow_freq
fut_service_df["Teal Trips"] = fut_service_df["Community"].isin(teal_list) * teal_trips
fut_service_df["Teal Freq"] = fut_service_df["Community"].isin(teal_list) * teal_freq
fut_service_df["Purple Trips"] = fut_service_df["Community"].isin(purple_list) * purple_trips
fut_service_df["Purple Freq"] = fut_service_df["Community"].isin(purple_list) * purple_freq
fut_service_df["East Grey Trips"] = fut_service_df["Community"].isin(east_grey_list) * east_grey_trips
fut_service_df["East Grey Freq"] = fut_service_df["Community"].isin(east_grey_list) * east_grey_freq
fut_service_df["West Grey Trips"] = fut_service_df["Community"].isin(west_grey_list) * west_grey_trips
fut_service_df["West Grey Freq"] = fut_service_df["Community"].isin(west_grey_list) * west_grey_freq
fut_service_df["Future Brentwood Trips"] = fut_service_df["Community"].isin(future_brentwood_list) * future_brentwood_trips
fut_service_df["Future Brentwood Freq"] = fut_service_df["Community"].isin(future_brentwood_list) * future_brentwood_freq
fut_service_df["Future Tuscany Trips"] = fut_service_df["Community"].isin(future_tuscany_list) * future_tuscany_trips
fut_service_df["Future Tuscany Freq"] = fut_service_df["Community"].isin(future_tuscany_list) * future_tuscany_freq


In [219]:
print(fut_service_df.shape)
fut_service_df.head()


(141, 23)


,Community,Red Trips,Red Freq,Blue Trips,Blue Freq,Green Trips,Green Freq,Orange Trips,Orange Freq,Yellow Trips,...,Purple Trips,Purple Freq,East Grey Trips,East Grey Freq,West Grey Trips,West Grey Freq,Future Brentwood Trips,Future Brentwood Freq,Future Tuscany Trips,Future Tuscany Freq
29,Acadia,226,10.0,0,0.0,0,0.0,0,0.0,0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
73,Airport,0,0.0,0,0.0,0,0.0,0,0.0,0,...,0,0.0,61,12.0,0,0.0,0,0.0,0,0.0
52,Altadore,0,0.0,0,0.0,0,0.0,0,0.0,75,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
79,Applewood Park,0,0.0,0,0.0,0,0.0,0,0.0,0,...,66,25.0,0,0.0,0,0.0,0,0.0,0,0.0
95,Arbour Lake,226,10.0,0,0.0,0,0.0,0,0.0,0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0


In [220]:
# Merge the two DataFrames on the 'Community' and 'Community Name' columns
merged_data = pd.merge(fut_service_df, comm_data, left_on='Community', right_on='Community Name', how='right')

# Update the 'Population' column in sup_fc with the corresponding values from comm_data
fut_service_df['Population'] = fut_service_df['Community'].apply(
    lambda x: merged_data.loc[merged_data['Community Name'] == x, 'Population in private households'].iloc[0]
    if x in merged_data['Community Name'].tolist() else 0
)

# dropping rows that have no population:
fut_service_df = fut_service_df[fut_service_df['Population'] != 0]

In [221]:
fut_service_df["Frequency Index"] = fut_service_df.filter(like="Freq").sum(axis=1)
fut_service_df['Total Trips'] = fut_service_df.filter(regex="Red Trips|Blue Trips|Green Trips").sum(axis=1)*train_capacity + fut_service_df.filter(regex="Orange Trips|Yellow Trips|Teal Trips|Yellow Trips|Purple Trips|East Grey Trips|West Grey Trips|Future Brentwood Trips| Future Tuscany Trips").sum(axis=1)*bus_capacity
fut_service_df["Capacity Index"] = fut_service_df['Total Trips']/fut_service_df.filter(like='Population').sum(axis=1)

In [222]:
min_freq = min(fut_service_df['Frequency Index'])
max_freq = max(fut_service_df['Frequency Index'])
min_cap = min(fut_service_df['Capacity Index'])
max_cap = max(fut_service_df['Capacity Index'])

In [223]:
fut_service_df['SS Frequency Index'] = (fut_service_df['Frequency Index']-min_freq)/(max_freq-min_freq)
fut_service_df['SS Capacity Index'] = (fut_service_df['Capacity Index']-min_cap)/(max_cap-min_cap)

In [190]:
fut_service_df

,Community,Red Trips,Red Freq,Blue Trips,Blue Freq,Green Trips,Green Freq,Orange Trips,Orange Freq,Yellow Trips,...,Future Brentwood Trips,Future Brentwood Freq,Future Tuscany Trips,Future Tuscany Freq,Population,Frequency Index,Total Trips,Capacity Index,SS Frequency Index,SS Capacity Index
29,Acadia,226,10.0,0,0.0,0,0.0,0,0.0,0,...,0,0.0,0,0.0,9915,22.0,139565,14.076147,0.318584,0.038927
52,Altadore,0,0.0,0,0.0,0,0.0,0,0.0,75,...,0,0.0,0,0.0,7290,9.5,4875,0.668724,0.097345,0.001849
79,Applewood Park,0,0.0,0,0.0,0,0.0,0,0.0,0,...,0,0.0,0,0.0,6830,25.0,4290,0.628111,0.371681,0.001737
95,Arbour Lake,226,10.0,0,0.0,0,0.0,0,0.0,0,...,0,0.0,0,0.0,10335,10.0,135600,13.120464,0.106195,0.036284
34,Aspen Woods,0,0.0,193,4.0,0,0.0,0,0.0,0,...,0,0.0,0,0.0,9435,4.0,115800,12.273450,0.000000,0.033942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,Whitehorn,0,0.0,193,4.0,0,0.0,98,7.0,0,...,0,0.0,0,0.0,11085,11.0,122170,11.021200,0.123894,0.030479
109,Willow Park,226,10.0,0,0.0,0,0.0,0,0.0,0,...,0,0.0,0,0.0,5090,10.0,135600,26.640472,0.106195,0.073674
0,Winston Heights/Mountview,0,0.0,0,0.0,0,0.0,98,7.0,0,...,0,0.0,0,0.0,3605,7.0,6370,1.766990,0.053097,0.004887
1,Woodbine,0,0.0,0,0.0,0,0.0,0,0.0,75,...,0,0.0,0,0.0,8745,9.5,4875,0.557461,0.097345,0.001542


# Combine the Supply index

In [253]:
current_cov = 'PTN service coverage.csv'
future_cov = 'Future Transit Service Coverage.csv'

In [254]:
curr_cov_file = pd.read_csv(data_path+current_cov)

In [255]:
# comm_bound_lower = comm_bound.copy()
# comm_bound_lower['name'] = comm_bound_lower['name'].str.lower()
# comm_data_lower = comm_data.copy()
# comm_data_lower['Community Name'] = comm_data_lower['Community Name'].str.lower()
# comm_z_lower = z_index.copy()
# comm_z_lower["Community Name"] = comm_z_lower['Community Name'].str.lower()

supply_df_upper = service_df.copy()
supply_df_upper['Community'] = supply_df_upper['Community'].str.upper()
fut_sup_upper = fut_service_df.copy()
fut_sup_upper['Community'] = fut_sup_upper["Community"].str.upper()

# Merge the DataFrames based on the lower case names
# merged_data = pd.merge(comm_data_lower, comm_bound_lower, left_on='Community Name', right_on='NAME', how='inner')
merged_data = pd.merge(supply_df_upper, curr_cov_file, how='right', left_on='Community', right_on='NAME')
future_merged = pd.merge(fut_sup_upper, curr_cov_file, how='right', left_on='Community', right_on='NAME')


In [256]:
print(merged_data.shape)
merged_data.head()

(199, 37)


,Community,Red Trips,Red Freq,Blue Trips,Blue Freq,Orange Trips,Orange Freq,Yellow Trips,Yellow Freq,Teal Trips,...,GLOBALID_GUID,CREATED_DT,MODIFIED_ACTION,MODIFIED_DT,Shape_Length,Shape_Area,sum_Area_SQUAREKILOMETERS,Polygon_Count,Service Coverage,SS_Ser_Cov
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,{C81B5298-6E3D-47E9-A4A7-BC3DAD6E58D7},10:01.0,MODIFIED ATTRIBUTE,04:44.0,9222.118881,4.300271e+06,0.000000,0,0.000000e+00,0.000000
1,ACADIA,226.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,61.0,...,{014B36D2-1114-40AA-B04B-E909E9AD701F},10:25.0,MODIFIED ATTRIBUTE,44:52.0,14488.117754,9.824282e+06,0.217225,3,3.053660e-07,0.060305
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,{2AE4E5D3-0A23-44B2-905D-E122B8D3D7E3},10:10.0,MODIFIED ATTRIBUTE,55:12.0,13076.831280,6.335590e+06,0.711608,5,7.891920e-07,0.155854
3,ALTADORE,0.0,0.0,0.0,0.0,0.0,0.0,75.0,9.5,0.0,...,{31AC3F2E-9AD8-4836-B5BD-5B95E9F4711B},10:10.0,MODIFIED ATTRIBUTE,52:51.0,12690.527548,5.377518e+06,0.000000,0,0.000000e+00,0.000000
4,APPLEWOOD PARK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,{40E94852-88AA-4A68-90FD-2DB494264583},10:30.0,MODIFIED ATTRIBUTE,04:36.0,8884.462043,3.925107e+06,0.158642,1,2.547700e-07,0.050313


In [257]:
final_supply = merged_data[['NAME','SS Frequency Index','SS Capacity Index','SS_Ser_Cov']]
final_future_supply = future_merged[['NAME','SS Frequency Index','SS Capacity Index','SS_Ser_Cov']]

In [258]:
final_supply = final_supply.rename(columns={'SS_Ser_Cov':"SS Coverage Index"})
final_supply = final_supply.fillna(0)
final_supply.shape

(199, 4)

In [259]:
final_future_supply = final_future_supply.rename(columns={'SS_Ser_Cov':"SS Coverage Index"})
final_future_supply = final_future_supply.fillna(0)
final_future_supply.shape

(199, 4)

In [260]:
final_supply['Average Supply Index'] = final_supply[['SS Frequency Index','SS Capacity Index','SS Coverage Index']].mean(axis=1)

In [261]:
final_future_supply['Average Supply Index'] = final_future_supply[['SS Frequency Index','SS Capacity Index','SS Coverage Index']].mean(axis=1)

In [262]:
final_supply.head()

,NAME,SS Frequency Index,SS Capacity Index,SS Coverage Index,Average Supply Index
0,ABBEYDALE,0.000000,0.000000,0.000000,0.000000
1,ACADIA,0.404494,0.062027,0.060305,0.175609
2,ALBERT PARK/RADISSON HEIGHTS,0.000000,0.000000,0.155854,0.051951
3,ALTADORE,0.123596,0.001630,0.000000,0.041742
4,APPLEWOOD PARK,0.471910,0.001447,0.050313,0.174557


In [263]:
final_future_supply.head()

,NAME,SS Frequency Index,SS Capacity Index,SS Coverage Index,Average Supply Index
0,ABBEYDALE,0.000000,0.000000,0.000000,0.000000
1,ACADIA,0.318584,0.038927,0.060305,0.139272
2,ALBERT PARK/RADISSON HEIGHTS,0.000000,0.000000,0.155854,0.051951
3,ALTADORE,0.097345,0.001849,0.000000,0.033065
4,APPLEWOOD PARK,0.371681,0.001737,0.050313,0.141244


# Merge Demand and Supply

In [335]:
demand_2 = "TotalDemandIndexStandardized.xlsx"
demand_df = pd.read_excel(data_path+demand_2)

In [336]:
demand_df['NAME'] = demand_df['NAME'].str.upper()

In [337]:
dns_merge = pd.merge(final_supply,demand_df, how='right', left_on='NAME', right_on='NAME')
dns_fut_merge = pd.merge(final_future_supply,demand_df, how='right', left_on='NAME', right_on='NAME')

In [338]:
print(dns_merge.shape)
filter = dns_merge[pd.isna(dns_merge['Average Supply Index'])]
print(filter.shape)

(199, 27)
(0, 27)


In [339]:
print(dns_fut_merge.shape)
filter = dns_fut_merge[pd.isna(dns_fut_merge['Average Supply Index'])]
print(filter.shape)

(199, 27)
(0, 27)


In [340]:
final_current_df = dns_merge[['NAME',"Low Income Index","Seniors Index","Rent Index","Total Community Index", "SS Frequency Index", "SS Capacity Index",'SS Coverage Index',"Standardized Total Community Index","Average Supply Index"]]
final_future_df = dns_merge[['NAME',"Low Income Index","Seniors Index","Rent Index","Total Community Index", "SS Frequency Index", "SS Capacity Index",'SS Coverage Index',"Standardized Total Community Index","Average Supply Index"]]

In [341]:
final_current_df = final_current_df.rename(columns={'Standardized Total Community Index': 'Community Demand Index'})
final_current_df = final_current_df.fillna(0)

In [342]:
final_future_df = final_future_df.rename(columns={'Standardized Total Community Index': 'Community Demand Index'})
final_future_df = final_future_df.fillna(0)

In [344]:
final_current_df["Transit Gap"] = final_current_df['Community Demand Index'] - final_current_df['Average Supply Index']

In [379]:
percentiles = final_current_df['Transit Gap'].quantile([0.2,0.4,0.6,0.8])
grade_boundaries = {
    'A': percentiles[0.8],  # Top 20% get an 'A'
    'B': percentiles[0.6],  # Next 20% get a 'B'
    'C': percentiles[0.4],  # Next 20% get a 'C'
    'D': percentiles[0.2],  # Next 20% get a 'D'
    'F': 0                   # Bottom 20% get an 'F'
}

In [380]:
final_current_df['Letter Grade'] = final_current_df['Transit Gap'].apply(assign_grade)

In [381]:
final_current_df

,NAME,Low Income Index,Seniors Index,Rent Index,Total Community Index,SS Frequency Index,SS Capacity Index,SS Coverage Index,Community Demand Index,Average Supply Index,Transit Gap,Letter Grade
0,ABBEYDALE,177.50,151.25,23.5125,417.2625,0.000000,0.000000,0.000000,0.134915,0.000000,0.134915,B
1,ACADIA,306.25,461.25,187.6500,1057.6500,0.404494,0.062027,0.060305,0.344485,0.175609,0.168876,B
2,ALBERT PARK/RADISSON HEIGHTS,291.25,188.75,154.7000,737.2000,0.000000,0.000000,0.155854,0.239616,0.051951,0.187665,A
3,ALTADORE,90.00,158.75,71.7750,350.5250,0.123596,0.001630,0.000000,0.113075,0.041742,0.071333,C
4,APPLEWOOD PARK,178.75,185.00,81.8625,525.6125,0.471910,0.001447,0.050313,0.170373,0.174557,-0.004184,F
...,...,...,...,...,...,...,...,...,...,...,...,...
194,WINSTON HEIGHTS/MOUNTVIEW,85.00,123.75,57.8000,300.3000,0.067416,0.006577,0.051708,0.096638,0.041900,0.054738,D
195,WOLF WILLOW,0.00,2.50,0.0000,5.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,F
196,WOODBINE,141.25,436.25,30.1500,638.9000,0.123596,0.001129,0.048671,0.207447,0.057798,0.149648,B
197,WOODLANDS,133.75,307.50,54.4500,529.4500,0.123596,0.002384,0.055991,0.171629,0.060657,0.110972,C


In [382]:
final_future_df["Transit Gap"] = final_future_df['Community Demand Index'] - final_future_df['Average Supply Index']

In [383]:
percentiles = final_future_df['Transit Gap'].quantile([0.2,0.4,0.6,0.8])
final_future_df["Letter Grade"] = final_future_df['Transit Gap'].apply(assign_grade)

In [384]:
final_future_df

,NAME,Low Income Index,Seniors Index,Rent Index,Total Community Index,SS Frequency Index,SS Capacity Index,SS Coverage Index,Community Demand Index,Average Supply Index,Transit Gap,Letter Grade
0,ABBEYDALE,177.50,151.25,23.5125,417.2625,0.000000,0.000000,0.000000,0.134915,0.000000,0.134915,B
1,ACADIA,306.25,461.25,187.6500,1057.6500,0.404494,0.062027,0.060305,0.344485,0.175609,0.168876,B
2,ALBERT PARK/RADISSON HEIGHTS,291.25,188.75,154.7000,737.2000,0.000000,0.000000,0.155854,0.239616,0.051951,0.187665,A
3,ALTADORE,90.00,158.75,71.7750,350.5250,0.123596,0.001630,0.000000,0.113075,0.041742,0.071333,C
4,APPLEWOOD PARK,178.75,185.00,81.8625,525.6125,0.471910,0.001447,0.050313,0.170373,0.174557,-0.004184,F
...,...,...,...,...,...,...,...,...,...,...,...,...
194,WINSTON HEIGHTS/MOUNTVIEW,85.00,123.75,57.8000,300.3000,0.067416,0.006577,0.051708,0.096638,0.041900,0.054738,D
195,WOLF WILLOW,0.00,2.50,0.0000,5.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,F
196,WOODBINE,141.25,436.25,30.1500,638.9000,0.123596,0.001129,0.048671,0.207447,0.057798,0.149648,B
197,WOODLANDS,133.75,307.50,54.4500,529.4500,0.123596,0.002384,0.055991,0.171629,0.060657,0.110972,C


# Add multipolygon to file

In [385]:
census_file = "Census by Community 2019_20240401.geojson"
comm_geo = gpd.read_file(data_path+census_file)

In [386]:
dns_bound_data = pd.merge(final_current_df, comm_geo, how='left', left_on='NAME', right_on='name')
current_dns = gpd.GeoDataFrame(dns_bound_data[['NAME',"Low Income Index","Seniors Index","Rent Index","Total Community Index", "SS Frequency Index", "SS Capacity Index",'SS Coverage Index',"Community Demand Index","Average Supply Index",'Transit Gap','Letter Grade','geometry']])

In [387]:
dns_bound_data = pd.merge(final_future_df, comm_geo, how='left', left_on='NAME', right_on='name')
future_dns = gpd.GeoDataFrame(dns_bound_data[['NAME',"Low Income Index","Seniors Index","Rent Index","Total Community Index", "SS Frequency Index", "SS Capacity Index",'SS Coverage Index',"Community Demand Index","Average Supply Index",'Transit Gap','Letter Grade','geometry']])

In [388]:
print(current_dns.shape)
current_dns.head()

(199, 13)


,NAME,Low Income Index,Seniors Index,Rent Index,Total Community Index,SS Frequency Index,SS Capacity Index,SS Coverage Index,Community Demand Index,Average Supply Index,Transit Gap,Letter Grade,geometry
0,ABBEYDALE,177.50,151.25,23.5125,417.2625,0.000000,0.000000,0.000000,0.134915,0.000000,0.134915,B,"MULTIPOLYGON (((-113.93512 51.06697, -113.9267..."
1,ACADIA,306.25,461.25,187.6500,1057.6500,0.404494,0.062027,0.060305,0.344485,0.175609,0.168876,B,"MULTIPOLYGON (((-114.05001 50.98078, -114.0492..."
2,ALBERT PARK/RADISSON HEIGHTS,291.25,188.75,154.7000,737.2000,0.000000,0.000000,0.155854,0.239616,0.051951,0.187665,A,"MULTIPOLYGON (((-113.98160 51.04515, -113.9816..."
3,ALTADORE,90.00,158.75,71.7750,350.5250,0.123596,0.001630,0.000000,0.113075,0.041742,0.071333,C,"MULTIPOLYGON (((-114.10055 51.02319, -114.0996..."
4,APPLEWOOD PARK,178.75,185.00,81.8625,525.6125,0.471910,0.001447,0.050313,0.170373,0.174557,-0.004184,F,"MULTIPOLYGON (((-113.92122 51.04520, -113.9212..."


In [389]:
print(future_dns.shape)
future_dns.head()

(199, 13)


,NAME,Low Income Index,Seniors Index,Rent Index,Total Community Index,SS Frequency Index,SS Capacity Index,SS Coverage Index,Community Demand Index,Average Supply Index,Transit Gap,Letter Grade,geometry
0,ABBEYDALE,177.50,151.25,23.5125,417.2625,0.000000,0.000000,0.000000,0.134915,0.000000,0.134915,B,"MULTIPOLYGON (((-113.93512 51.06697, -113.9267..."
1,ACADIA,306.25,461.25,187.6500,1057.6500,0.404494,0.062027,0.060305,0.344485,0.175609,0.168876,B,"MULTIPOLYGON (((-114.05001 50.98078, -114.0492..."
2,ALBERT PARK/RADISSON HEIGHTS,291.25,188.75,154.7000,737.2000,0.000000,0.000000,0.155854,0.239616,0.051951,0.187665,A,"MULTIPOLYGON (((-113.98160 51.04515, -113.9816..."
3,ALTADORE,90.00,158.75,71.7750,350.5250,0.123596,0.001630,0.000000,0.113075,0.041742,0.071333,C,"MULTIPOLYGON (((-114.10055 51.02319, -114.0996..."
4,APPLEWOOD PARK,178.75,185.00,81.8625,525.6125,0.471910,0.001447,0.050313,0.170373,0.174557,-0.004184,F,"MULTIPOLYGON (((-113.92122 51.04520, -113.9212..."
